In [12]:
from tensorflow import keras
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
import xlrd
from ipynb.fs.full.fullDataExtraction import extarctChannelNames
from ipynb.fs.full.feature_selection import get_features
from mne_features.feature_extraction import extract_features
from ipynb.fs.full.LabelTesting import makeOutThreeArr
from ipynb.fs.full.fullDataExtraction import compute_diffEnt
import pandas as pd

In [2]:
model = keras.models.load_model("emotionPredictionModel")

In [3]:
matfile=loadmat('1_20131107.mat')
del matfile["__header__"]
del matfile["__version__"]
del matfile["__globals__"]
clip_info={}
indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
keyName=list(matfile.keys())[0][:-1]
channelNames=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
info=mne.create_info(channelNames,200,'eeg')
for ind,i in enumerate(indexs):
    rawData=matfile[keyName + str(i)]
    clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)

In [8]:
sliced_epochs={}
for clip in clip_info:
    sliced_epochs[clip]= mne.make_fixed_length_epochs(clip_info[clip], duration=1, preload=True,verbose=0)

In [13]:
features=['hjorth_mobility', ('diffEnt',compute_diffEnt), 'rms', 'skewness']

In [14]:
combined = np.zeros((1,len(features)*62))
for cut in sliced_epochs:
    epoch_array=mne.Epochs.get_data(sliced_epochs[cut])
    extracted_data=extract_features(epoch_array,200,features)
    combined = np.vstack((combined,extracted_data))
combined = np.delete(combined, 0, axis=0)

In [18]:
combined.shape

(3394, 248)

In [15]:
y_pred = model.predict(combined)
y_pred = (y_pred > 0.5).astype(int)

107/107 [==============================] - 1s 1ms/step


In [16]:
y_pred

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0]])